Notebook Setup

In [79]:
import warnings
warnings.filterwarnings('ignore')

In [80]:
#import dependencies
import requests
from pprint import pprint
from requests.auth import HTTPBasicAuth
import pandas as pd

%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import scipy.stats as stats

#authentication flow
client_id = "4b454f446d1f479082eec9de1c27a4a8"
client_secret = "e639450422594948b17c46d1a82f4882"

url = "https://accounts.spotify.com/api/token?grant_type=client_credentials"
headers = {'content-type': 'application/x-www-form-urlencoded'}

try:
   response = requests.post(url, headers=headers, auth=(client_id, client_secret))
   json = response.json()
   print(json)
   access_token = json["access_token"]
   print(access_token)
except:
   print("Something went wrong with the request")

{'access_token': 'BQBcLPTiuIaK-s5-KoJcQ-vV04yvUDeN4on2M7LaSV_kUv7Qi33QNtbG-R5nIrQSi218lECZ_2sYYEBI0-o', 'token_type': 'Bearer', 'expires_in': 3600, 'scope': ''}
BQBcLPTiuIaK-s5-KoJcQ-vV04yvUDeN4on2M7LaSV_kUv7Qi33QNtbG-R5nIrQSi218lECZ_2sYYEBI0-o


Collect Data

In [81]:
# https://api.spotify.com/v1/artists/{id}

#This is for US Top 50 playlist
playlist_id = "37i9dQZEVXbLRQDuF5jeBp"
#spotify:user:spotifycharts:playlist:37i9dQZEVXbLRQDuF5jeBp

url = f"https://api.spotify.com/v1/playlists/{playlist_id}"

headers = {'authorization': f"Bearer {access_token}"}
response = requests.get(url, headers=headers)

#pprint(response.json())
#print(json.dumps(newrelease, sort_keys=True, indent=4))
jj = response.json()
#pprint(jj)

In [82]:
"""for playlist in pl: 
    url = f"https://api.spotify.com/v1/playlists/{playlist}"

    headers = {'authorization': f"Bearer {access_token}"}
    response = requests.get(url, headers=headers)

    #pprint(response.json())
    #print(json.dumps(newrelease, sort_keys=True, indent=4))
    jj = response.json()
    
    for x in jj:
        pprint(x['tracks'])"""

'for playlist in pl: \n    url = f"https://api.spotify.com/v1/playlists/{playlist}"\n\n    headers = {\'authorization\': f"Bearer {access_token}"}\n    response = requests.get(url, headers=headers)\n\n    #pprint(response.json())\n    #print(json.dumps(newrelease, sort_keys=True, indent=4))\n    jj = response.json()\n    \n    for x in jj:\n        pprint(x[\'tracks\'])'

In [83]:
#test check: to grab album uri
#jj['tracks']['items'][0]['track']['album']['uri'][14:])
#test check to grab artist uri
pprint(jj['tracks']['items'][0]['track']['album']['artists'][0]["uri"][15:])

'66CXWjxzNUsdJxJ2JdwvnR'


In [84]:
songs= jj['tracks']['items']

In [85]:
#Create lists for relevant data & then organize into a dataframe
ids = []
artists = []
tracks = []
releasedate = []
audiofeatures = []
popularity = []
for x in songs:
    idsong = x["track"]["id"]
    ids.append(idsong)
    artist = x['track']['album']['artists'][0]['name']
    artists.append(artist)
    track = x['track']['name']
    tracks.append(track)
    rd = x["track"]["album"]["release_date"]
    releasedate.append(rd)
    poppin = x['track']['popularity']
    popularity.append(poppin)
    url = f"https://api.spotify.com/v1/audio-features/{idsong}"
    headers = {'authorization': f"Bearer {access_token}"}
    response = requests.get(url, headers=headers)
    features= response.json()
    audiofeatures.append(features)
#pprint(features)

In [86]:
#looking for genre of each artist via for loop
artid = []
genre = []
for x in songs:
    ida = x['track']['album']['artists'][0]["uri"][15:]
    artid.append(ida)
    url = f"https://api.spotify.com/v1/artists/{ida}"
    headers = {'authorization': f"Bearer {access_token}"}
    response = requests.get(url, headers=headers)
    gen= response.json()
    genr = gen["genres"]
    genre.append(genr)

In [87]:
#separate the dictionary into lists
afkeys = [list(x.keys()) for x in audiofeatures]
afvalues = [list(x.values()) for x in audiofeatures]

#create a dataframe for audio features
af = pd.DataFrame(afvalues, columns=afkeys[0])
#create a dataframe for track info
tdata = pd.DataFrame(list(zip(artists, artid, tracks, ids, releasedate, popularity, genre)),columns=["artists", "artist id", "tracks", "track ids", "released", "popularity", "genre"])

In [88]:
tdata.head()

,artists,artist id,tracks,track ids,released,popularity,genre
0,Ariana Grande,66CXWjxzNUsdJxJ2JdwvnR,7 rings,6ocbgoVGwYJhOv1GgI9NsF,2019-02-08,92,"[dance pop, pop, post-teen pop]"
1,Ariana Grande,66CXWjxzNUsdJxJ2JdwvnR,"break up with your girlfriend, i'm bored",4kV4N9D1iKVxx1KLvtTpjS,2019-02-08,97,"[dance pop, pop, post-teen pop]"
2,J. Cole,6l3HvQ5sa6mXTsMTB19rO5,MIDDLE CHILD,2JvzF1RMd7lE3KmFlsyZD8,2019-01-23,95,"[conscious hip hop, nc hip hop, pop rap, rap]"
3,Post Malone,246dkjvS1zLTtiykXe5h60,Wow.,6MWtB6iiXyIwun0YzU6DFP,2018-12-24,98,"[pop, rap]"
4,Various Artists,0LyfQWJT6nXafLPZqxe9Of,Sunflower - Spider-Man: Into the Spider-Verse,7dbka99KTWke5o9hRp0JoB,2019-02-22,66,[]


In [89]:
#combine the two dataframes 
data = pd.concat([tdata, af], axis=1, sort=False)

#clean data by removing unneccesary columns
data.drop(["uri", "track_href", "analysis_url", "id"],axis = 1, inplace = True)
data

,artists,artist id,tracks,track ids,released,popularity,genre,danceability,energy,key,...,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,duration_ms,time_signature
0,Ariana Grande,66CXWjxzNUsdJxJ2JdwvnR,7 rings,6ocbgoVGwYJhOv1GgI9NsF,2019-02-08,92,"[dance pop, pop, post-teen pop]",0.771,0.320,1,...,0,0.3160,0.60100,0.000000,0.0885,0.312,139.844,audio_features,178627,4
1,Ariana Grande,66CXWjxzNUsdJxJ2JdwvnR,"break up with your girlfriend, i'm bored",4kV4N9D1iKVxx1KLvtTpjS,2019-02-08,97,"[dance pop, pop, post-teen pop]",0.726,0.545,5,...,0,0.1020,0.03780,0.000000,0.1080,0.348,169.991,audio_features,190440,4
2,J. Cole,6l3HvQ5sa6mXTsMTB19rO5,MIDDLE CHILD,2JvzF1RMd7lE3KmFlsyZD8,2019-01-23,95,"[conscious hip hop, nc hip hop, pop rap, rap]",0.837,0.364,8,...,1,0.2760,0.14900,0.000000,0.2710,0.463,123.984,audio_features,213594,4
3,Post Malone,246dkjvS1zLTtiykXe5h60,Wow.,6MWtB6iiXyIwun0YzU6DFP,2018-12-24,98,"[pop, rap]",0.833,0.539,11,...,0,0.1780,0.16300,0.000002,0.1010,0.385,99.947,audio_features,149520,4
4,Various Artists,0LyfQWJT6nXafLPZqxe9Of,Sunflower - Spider-Man: Into the Spider-Verse,7dbka99KTWke5o9hRp0JoB,2019-02-22,66,[],0.761,0.479,2,...,1,0.0466,0.55200,0.000000,0.0704,0.912,89.913,audio_features,158040,4
5,Lady Gaga,1HY2Jd0NmPuamShAr6KMms,Shallow,2VxeLyX666F8uXCJ0dZF8B,2018-10-05,92,"[dance pop, pop]",0.572,0.385,7,...,1,0.0308,0.37100,0.000000,0.2310,0.323,95.799,audio_features,215733,4
6,YNW Melly,1cNDP5yjU5vjeR8qMf4grg,Murder On My Mind,7eBqSVxrzQZtK2mmgRG6lC,2018-08-03,88,"[rap, trap music]",0.759,0.730,0,...,0,0.0516,0.14500,0.000003,0.1100,0.740,115.007,audio_features,268434,4
7,Juice WRLD,4MCBfE4596Uoi2O4DtmEMz,Robbery,44ShclXTOOnYtt5CnCDeGV,2019-02-13,84,[chicago rap],0.708,0.691,2,...,1,0.0442,0.34200,0.000000,0.2220,0.535,79.990,audio_features,240527,4
8,Meek Mill,20sxb77xiYeusSH8cVdatc,Going Bad (feat. Drake),2IRZnDFmlqMuOrYOLnZZyc,2018-11-30,94,"[hip hop, philly rap, pop, pop rap, rap, trap ...",0.889,0.496,4,...,0,0.0905,0.25900,0.000000,0.2520,0.544,86.003,audio_features,180522,4
9,21 Savage,1URnnhqYAYcrqrcwql10ft,a lot,2t8yVaLvJ0RenpXUIAC52d,2018-12-21,92,"[atl hip hop, pop, pop rap, rap, trap music]",0.837,0.636,1,...,1,0.0860,0.03950,0.001250,0.3420,0.274,145.972,audio_features,288624,4


In [90]:
data.describe()

,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,50.000000,50.000000,50.000000,50.00000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000,50.000000
mean,86.980000,0.745520,0.557760,5.56000,-6.926800,0.620000,0.155998,0.266033,0.011106,0.172108,0.425760,121.384420,202788.740000,3.980000
std,8.582112,0.125958,0.122978,3.50603,2.305502,0.490314,0.119164,0.205255,0.052173,0.096829,0.181713,28.684726,38350.040776,0.141421
min,66.000000,0.448000,0.314000,0.00000,-14.505000,0.000000,0.027400,0.001950,0.000000,0.070400,0.111000,74.972000,129264.000000,3.000000
25%,84.500000,0.665750,0.475250,2.00000,-8.015750,0.000000,0.066675,0.083125,0.000000,0.106500,0.325250,98.519000,178832.500000,4.000000
50%,90.000000,0.749500,0.564500,6.00000,-6.706500,1.000000,0.100250,0.217000,0.000000,0.126000,0.380500,122.015000,199188.500000,4.000000
75%,92.000000,0.837000,0.656000,8.00000,-5.292000,1.000000,0.242500,0.387500,0.000003,0.222000,0.534750,144.523500,225793.250000,4.000000
max,98.000000,0.950000,0.792000,11.00000,-2.749000,1.000000,0.565000,0.787000,0.334000,0.552000,0.912000,202.006000,312820.000000,4.000000


In [91]:
#wanted to find a list of all playlists in the Top 50 Charts by Country category, but such category does not exist in the API.
#alternate resolution: manually copy and paste the playlist ids from spotify
#alternate resolution: instead of comparing data from all countries, we can analze the GLOBAL TOP 50 playlist vs GLOBAL VIRAL 50 playlist

url = f"https://api.spotify.com/v1/browse/categories/charts/playlists"
headers = {'authorization': f"Bearer {access_token}"}
response = requests.get(url, headers=headers)
cate= response.json()
pprint(cate)

{'error': {'message': "Specified id doesn't exist", 'status': 404}}


In [92]:
#data.to_csv("top50us.csv", encoding='utf-8', index=False)

In [93]:
uspop = data.popularity.mean()
brazilpop= 76.68

In [94]:
stats.ttest_ind(uspop,brazilpop, equal_var=False)

Ttest_indResult(statistic=nan, pvalue=nan)